In [1]:
import torch
from torch_geometric.data import Data

In [2]:
from torch import tensor

In [3]:
a =  tensor([14, 30,  9, 26,  3, 22, 27, 26, 20, 18,  0])

RuntimeError: CUDA error: the provided PTX was compiled with an unsupported toolchain.

In [15]:
a1 = a.unsqueeze(1)

In [17]:
a1

tensor([[14],
        [30],
        [ 9],
        [26],
        [ 3],
        [22],
        [27],
        [26],
        [20],
        [18],
        [ 0]])

In [26]:
b = tensor([[ 7, 15,  6],
         [ 0,  4,  8],
         [ 1,  3,  4],
         [ 2,  3,  3],
         [ 3,  4,  5],
         [ 3,  1,  4],
         [ 0,  4,  5],
         [ 9,  4,  6],
         [ 7,  1,  4],
         [ 8,  2,  3],
         [ 2,  4,  9],
         [ 0,  0, 10],
         [ 1,  0, 10],
         [ 2,  0, 10],
         [ 3,  0, 10],
         [ 4,  0, 10],
         [ 5,  0, 10],
         [ 6,  0, 10],
         [ 7,  0, 10],
         [ 8,  0, 10],
         [ 9,  0, 10]])

In [33]:
b[:,[0,2]]

tensor([[ 7,  6],
        [ 0,  8],
        [ 1,  4],
        [ 2,  3],
        [ 3,  5],
        [ 3,  4],
        [ 0,  5],
        [ 9,  6],
        [ 7,  4],
        [ 8,  3],
        [ 2,  9],
        [ 0, 10],
        [ 1, 10],
        [ 2, 10],
        [ 3, 10],
        [ 4, 10],
        [ 5, 10],
        [ 6, 10],
        [ 7, 10],
        [ 8, 10],
        [ 9, 10]])

In [34]:
from torch_scatter import scatter_mean
from torch_geometric.datasets import TUDataset
from torch_geometric.data import DataLoader

dataset = TUDataset(root='/tmp/ENZYMES', name='ENZYMES', use_node_attr=True)
loader = DataLoader(dataset, batch_size=32, shuffle=True)

Extracting /tmp/ENZYMES/ENZYMES/ENZYMES.zip
Processing...
Done!


In [35]:
for data in loader:
    print(data.x.shape)
    print(data.batch.shape)
    
    x = scatter_mean(data.x, data.batch, dim=0)
    break

torch.Size([1016, 21])
torch.Size([1016])
